In [3]:
import glob
import os
import pickle
import sys
from pathlib import Path
import random
from tqdm import tqdm

import torchvision.models as models
from torch.nn import Module, Dropout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torchvision

#from pytorchtools import EarlyStopping

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from torchvision import datasets, models, transforms
import time
import logging

#from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import KFold
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV



In [4]:
train_transforms = transforms.Compose([
                                    transforms.Resize(224),
                                    transforms.RandomVerticalFlip(),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ColorJitter(brightness=0, contrast=0, saturation=1, hue=.5),
                                    transforms.ToTensor()])


val_transforms= transforms.Compose([
                                    transforms.Resize(224),
                                    transforms.ToTensor()])



# cross validaion with 5 folders on the train 
train_dataset = datasets.ImageFolder('/path/to/your/tiles', train_transforms) 

# cross validaion with 5 folders on the val 
val_dataset = datasets.ImageFolder('/path/to/your/tiles', val_transforms) 


batch_size = 48
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
valid_loader= torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=16, shuffle=True)

                         

/Users/eudari01/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
dropout=0.5

In [ ]:
class VGG_19_Regression(Module):
    
    def __init__(self,dropout=0.8):
        
        
        model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19_bn',weights = None)
        
        super(VGG_19_Regression,self).__init__()
        self.model= model
        self.Regression = nn.Linear(1000,1)
        self.Batch=nn.BatchNorm1d(1000)
        #self.Flatten=nn.Flatten()
        self.dropout = Dropout(float(dropout))
        
           
         
    def forward(self, out):
        
      
        x= self.model(out)
        
        x=self.Batch(x)
        
        x = x.view(x.size(0), -1) # flatten
        
        x = self.dropout(x)
        
        
        
        x =self.Regression(x) 
        
        
        return x

In [9]:
Model= VGG_19_Regression()
MSE = torch.nn.MSELoss()
optimizer = torch.optim.SGD(Model.parameters(), lr=0.001,momentum=0.9, weight_decay=1e-5)


Using cache found in /Users/eudari01/.cache/torch/hub/pytorch_vision_v0.10.0


In [10]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    Model = nn.DataParallel(Model)
    
torch.manual_seed(42)
#Model.to(device)    

In [12]:
#train_accuracy=[]
train_loss = []
val_loss = []
r2_score_train=[]
r2_score_val= []




def training(epochs):

      
        
                
                    
        Model.train()              
        epoch_loss = 0.0
        running_loss_train=0.0
   
        for inputs, label in tqdm(train_loader):
        
                  inputs= inputs.to(device)
                  label = label.to(device)
                
                
                  label= label.to(torch.float32)
    
                  label = label.unsqueeze(1)
                  optimizer.zero_grad()
                  
                  output = Model(inputs)
       
                  loss = MSE(output,label)

            
                  loss.backward()
                  optimizer.step()
                
            #Calculate prediction
                  prediction_train=output.argmax(dim=1)

                
        
                  running_loss_train =+ loss.item()* inputs.size(0) 
            #Epoch
            
                  epoch_loss += running_loss_train / len(train_loader)
                  r2_score_train=r2_score(label,prediction_train)
                
  
        
        # save the model after training 
        torch.save(Model(), "")
            
        Model.eval()
        epoch_val_loss = 0.0
        running_loss_val= 0.0
       
        
        with torch.no_grad():
        
            for inputs, label in valid_loader:
                
                        
                        inputs= inputs.to(device)
                        label = label.to(device)
                        
                        label= label.to(torch.float32)
                        label = label.unsqueeze(1)

                        output = Model(inputs)
                
                        loss = MSE(output, label)
                
              
                        prediction_val= output.argmax(dim=1)
                    
                
                #mse_val=mean_squared_error(label,prediction_val)
                
                #val_acc = (prediction_val == label).float().mean()
                
                        running_loss_val =+ loss.item()* inputs.size(0) 
               
                #Epoch  
                    
                        epoch_val_loss += running_loss_val / len(valid_loader)
                        r2_score_val= r2_score( label,prediction_val)
                        
        #to resset the train and validation for each split           
   
     
                    
                        
        val_loss.append(epoch_val_loss)
        train_loss.append(epoch_loss)
        
  

        
        print(
            f"Epoch : {epoch+1} - train_loss : {epoch_loss:.4f} - val_loss : {epoch_val_loss:.4f} -r2_score_train:{r2_score_train:.4f}-r2_score_val:{r2_score_val:.4f}\n")
        
                


        

        
        return epoch_loss, epoch_val_loss,r2_score_train,r2_score_val



In [13]:
epochs=15
for epoch in range(1,epochs+1):
     results_df=training(epoch)
     
     results_df.to_csv('/data/results_optuna.csv')


100%|███████████████████████████████████████████| 13/13 [02:19<00:00, 10.69s/it]


Epoch : 2 - train_loss : 30.8897 - val_loss : 16.4911 -Split: 0



In [14]:
import optuna

In [30]:
def objective (trial):
    
    
    param={   
    'batch_size': trial.suggest_int('batch_size',10,60),
    'lr': trial.suggest_loguniform('lr',1e-7, 1e-1),
    'dropout': trial.suggest_loguniform('dropout',0.3,0.9),
    'momentum' : trial.suggest_loguniform('momentum',0.1,0.9),
     'weight_decay': trial.suggest_loguniform('weight_decay',1e-5, 1e-1)
    }
    
    
    val_transforms= transforms.Compose([
                                    transforms.Resize(224),
                                    transforms.ToTensor()])

    val_dataset = datasets.ImageFolder('/path/data/validation', val_transforms) 

    valid_loader= torch.utils.data.DataLoader(val_dataset, param["batch_size"], num_workers=16, shuffle=True)
    
    optimizer = torch.optim.SGD(Model.parameters(), param["lr"],param["momentum"],param["weight_decay"])
    
    
    #Model=Model(param["dropout"])  
    Model.eval()
    MSE = torch.nn.MSELoss()
 
        
        

    
        #cross validation 
  
                
        
    epoch_val_loss = 0.0
    running_loss_val= 0.0
          
        
    for data, label in valid_loader:
        
            label= label.to(torch.float32)
                  
            label = torch.unsqueeze(label,1)
                        
            output = Model(data)
                
            loss = MSE(output, label)
                
            prediction_val= output.argmax(dim=1)
                
             
                
            running_loss_val =+ loss.item()* data.size(0) 
                   
               
                #Epoch
               
            epoch_val_loss += running_loss_val / len(valid_loader)
                        
            val_loss=[]
            val_loss.append(epoch_val_loss)
                
                        
    return epoch_val_loss

In [31]:
    sampler = optuna.samplers.TPESampler()

    study = optuna.create_study(direction='minimize',sampler=sampler)

    study.optimize(objective, n_trials=1)

    trial = study.best_trial


    print("Best hyperparameters: {}".format(trial.params))
    
    #print("Best hyperparameters2: {}".format(study.best_params))
    print("\nBest MSE : {}".format(study.best_value) )
    

[I 2022-08-29 11:13:10,464] A new study created in memory with name: no-name-087c3127-95e9-4a01-bc5f-94bd1bf30eea
[W 2022-08-29 11:13:10,469] Trial 0 failed because of the following error: FileNotFoundError(2, 'No such file or directory')
Traceback (most recent call last):
  File "/Users/eudari01/anaconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_61127/1359690696.py", line 17, in objective
    val_dataset = datasets.ImageFolder('/data/tiles_CIN/val/', val_transforms)
  File "/Users/eudari01/anaconda3/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 310, in __init__
    super().__init__(
  File "/Users/eudari01/anaconda3/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 145, in __init__
    classes, class_to_idx = self.find_classes(self.root)
  File "/Users/eudari01/anaconda3/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 219, in find_classe

FileNotFoundError: [Errno 2] No such file or directory: '/data/tiles_CIN/val/'